In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import pickle
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
from natsort import natsorted

# Add function path
original_path = os.getcwd()
os.chdir(original_path)
function_path = './functions/'
sys.path.append(function_path)

# Import custom functions
from analysis_function import *
from kcc_constrain_function import *
from Plot_function import *
from f1_0_H_sum_multi_region_constrain import *
from f_IMP import *
from f_schemes import *

In [3]:
path = './saved_data/'

def load_xr_pickle(filename):
    with open(filename, "rb") as f:
        return pickle.load(f)

obs_all = load_xr_pickle(path + '0.1.HadCRUT5.Tas.anomalies.46AR6regions_7cont_1glob_175years_1850-2024.pkl')

obs_200runs_all = load_xr_pickle(path + '0.2.HadCRUT5.200runs.nonmasked.Tas.anomalies.46AR6regions_7cont_1glob_175years_1850-2024.pkl')

mod_all = load_xr_pickle(path + '0.4.Smoothed_His-ALL.25mods.mean.nonmasked.Tas.anomalies.46AR6regions_7cont_1glob_171years_1850-2020.pkl')

ln_mod_all = load_xr_pickle(path + '0.3.Large_ensembles.320runs.nonmasked.Tas.anomalies.46AR6regions_7cont_1glob_251years_1850-2100.pkl')

mod_45_pseudo = load_xr_pickle(path + '0.5.pseudo-model.15mod.run1-3.nonmasked.Tas.anomalies.46AR6regions_7cont_1glob_176years_1850-2025.pkl')

print_xarray_info(obs_all, obs_200runs_all, mod_all, ln_mod_all, mod_45_pseudo)


Array 1:
  Sizes: Frozen({'year': 175, 'region': 54})
  Coords: ['realization', 'year', 'region', 'abbrevs', 'names']

Array 2:
  Sizes: Frozen({'runs': 200, 'year': 175, 'region': 54})
  Coords: ['runs', 'year', 'region', 'abbrevs', 'names']

Array 3:
  Sizes: Frozen({'forcing': 1, 'model_name': 25, 'year': 176, 'region': 54})
  Coords: ['model_name', 'region', 'year', 'abbrevs', 'names', 'forcing']

Array 4:
  Sizes: Frozen({'model_run': 320, 'year': 251, 'region': 54})
  Coords: ['year', 'model_run', 'model_name', 'region', 'abbrevs', 'names']

Array 5:
  Sizes: Frozen({'model_run': 45, 'year': 176, 'region': 54})
  Coords: ['model_run', 'year', 'model_name', 'region', 'abbrevs', 'names']


In [76]:
# -----------------------------------------
# Define constraint schemes
# -----------------------------------------

region_groups = {
    'North and Central America': ['GIC', 'NWN', 'NEN', 'WNA', 'CNA', 'ENA', 'NCA', 'SCA', 'CAR'],
    'South America': ['NWS', 'NSA', 'NES', 'SAM', 'SWS', 'SES', 'SSA'],
    'Europe': ['NEU', 'WCE', 'EEU', 'MED'],
    'Africa': ['SAH', 'WAF', 'CAF', 'NEAF', 'SEAF', 'WSAF', 'ESAF', 'MDG'],
    'Asia': ['RAR', 'WSB', 'ESB', 'RFE', 'WCA', 'ECA', 'TIB', 'EAS', 'ARP', 'SAS'],
    'Australasia': ['SEA', 'NAU', 'CAU', 'EAU', 'SAU', 'NZ'],
    'Antarctica': ['EAN', 'WAN'],
    # 'Global land': ['LSAT']
}

continent_abbrevs = ['NA', 'SA', 'EU', 'AF', 'Asia', 'AU', 'ANT']

scheme_dict = {
    "single_region": generate_single_region_constraint_pairs(region_groups),
    "region_within_continent": generate_continent_allregion_constraint_pairs(region_groups),
    "continent_mean": generate_continent_mean_constraint_pairs(region_groups, continent_abbrevs),
    "global_mean": generate_global_target_constraint_pairs(region_groups, global_abbrev='LSAT'),
    "global_region": generate_global_regional_target_constraint_pairs(region_groups, global_abbrev='LSAT'),
    "global_continent": generate_continent_and_global_constraint_pairs(region_groups, continent_abbrevs, global_abbrev='LSAT')
}

In [78]:
pseudo_45 = mod_45_pseudo.where(~xr.ufuncs.isnan(obs_all)).rename({'model_run': 'pseudo'})
mod_all_repeat_45 = mo_repeat(mod_all, pseudo_45)
full_pseudo_45 = mod_45_pseudo.rename({'model_run': 'pseudo'})


In [79]:
pseudo = pseudo_45
obs_200runs = obs_200runs_all
ln_mod = ln_mod_all
mod_his_repeat = mod_all_repeat_45
mod_da_repeat = mod_all_repeat_45
pseudo_ar6 = pseudo_45

region_names = list(mod_da_repeat.names.values)
forcing_list = mod_da_repeat.forcing  # xarray.DataArray
his_forcing = ['ALL']
constrain_forcing_names=['ALL']

region_abbrevs = list(mod_da_repeat.abbrevs.values)
region_slices_name = get_region_slices_from_groups(region_groups, region_abbrevs)

region_subset = {k: region_slices_name[k] for k in ['Europe']}
constrain_func = constrain_sum_reg

# -----------------------------------------
# Define time periods
# -----------------------------------------

ref_period = (1850, 1900)
# ref_period2 = (1961, 1990)

warming_target_period1 = (2016, 2025)
# warming_target_period2 = (1996, 2025)


all_prior_results = {}
all_post_results = {}

for scheme_name, scheme_pairs in scheme_dict.items():

# first_item = list(scheme_dict.items())[:1]
# for scheme_name, scheme_pairs in first_item:    
    print(f"\nRunning scheme: {scheme_name}")
    
    prior_results_1625, post_results_1625 = run_all_pseudos(
        scheme_pairs,
        constrain_func,
        pseudo,
        obs_200runs,
        ln_mod,
        mod_his_repeat,
        mod_da_repeat,
        pseudo_ar6,
        region_names,
        forcing_list,
        his_forcing,
        constrain_forcing_names,
        region_slices=region_slices_name,
        pseudo_ids= list(range(len(pseudo['pseudo']))),
        ref_period=ref_period,
        warming_target_period=warming_target_period1,
        uncertainty_ref_period=(1850, 2025),
        obs_adjust_ref_period=(1961, 2023),
        calc_smoothed=False)
    
    all_prior_results[scheme_name] = prior_results_1625
    all_post_results[scheme_name] = post_results_1625


full_pseudo_45_rename = full_pseudo_45.assign_coords(
    region = ("region", full_pseudo_45["abbrevs"].values)
)

prior_all_schemes, post_all_schemes, pseudo_warming = prepare_prior_post_with_pseudo(
    all_prior_results, all_post_results, full_pseudo_45_rename,
    warming_target_period1, ref_period
)


Running scheme: single_region

Running pseudo 0
  Region slice: North and Central America
  Region slice: South America
  Region slice: Europe
  Region slice: Africa
  Region slice: Asia
  Region slice: Australasia
  Region slice: Antarctica

Running pseudo 1
  Region slice: North and Central America
  Region slice: South America
  Region slice: Europe
  Region slice: Africa
  Region slice: Asia
  Region slice: Australasia
  Region slice: Antarctica

Running pseudo 2
  Region slice: North and Central America
  Region slice: South America
  Region slice: Europe
  Region slice: Africa
  Region slice: Asia
  Region slice: Australasia
  Region slice: Antarctica

Running pseudo 3
  Region slice: North and Central America
  Region slice: South America
  Region slice: Europe
  Region slice: Africa
  Region slice: Asia
  Region slice: Australasia
  Region slice: Antarctica

Running pseudo 4
  Region slice: North and Central America
  Region slice: South America
  Region slice: Europe
  Region

In [82]:
post_all_schemes

<xarray.DataArray 'post_warming' (scheme: 6, pseudo: 45, region: 46,
                                  forcing: 1, quantile: 3)> Size: 298kB
array([[[[[2.25541046, 0.54084972, 3.96997121]],

         [[2.11989228, 1.34652631, 2.89325826]],

         [[2.71020652, 1.54447452, 3.87593852]],

         ...,

         [[1.123903  , 0.9677114 , 1.28009459]],

         [[1.3331155 , 0.71995835, 1.94627265]],

         [[1.46730665, 0.48326033, 2.45135296]]],


        [[[2.25541046, 0.54084972, 3.96997121]],

         [[2.28077082, 1.50740485, 3.0541368 ]],

         [[2.71020652, 1.54447452, 3.87593852]],
...

         [[1.28971668, 0.86004508, 1.71938829]],

         [[1.17035981, 0.3249295 , 2.01579011]]],


        [[[2.86469359, 2.06897198, 3.66041519]],

         [[2.70139976, 2.22136763, 3.1814319 ]],

         [[2.92383729, 2.38974566, 3.45792892]],

         ...,

         [[1.09407187, 0.88544229, 1.30270146]],

         [[1.40338394, 0.97371233, 1.83305554]],

         [[1.32347649, 0.47804619, 2.16890679]]]]],
      shape=(6, 45, 46, 1, 3))
Coordinates:
  * pseudo        (pseudo) object 360B 'ACCESS-CM2_run1' ... 'UKESM1-0-LL_run3'
  * quantile      (quantile) <U4 48B 'mean' '5th' '95th'
  * forcing       (forcing) <U3 12B 'ALL'
  * region        (region) <U4 736B 'GIC' 'NWN' 'NEN' 'WNA' ... 'NZ' 'EAN' 'WAN'
    region_group  (region) <U25 5kB 'North and Central America' ... 'Antarctica'
  * scheme        (scheme) <U23 552B 'single_region' ... 'global_continent'

In [81]:
ds_results = xr.Dataset({
    "prior_all_schemes": prior_all_schemes,
    "post_all_schemes": post_all_schemes,
    "pseudo_warming": pseudo_warming,
})

save_path = './saved_data/'
name = '1.0_IPM_Prior_posterier_Pseudo_warming_period_2016-2025_ref1850-1900.pkl'

with open(save_path + name, 'wb') as wi:
	pickle.dump(ds_results, wi)